# Notebook 01: Data Loading, Validation, and Cleaning

---

## Executive Summary

This notebook serves as the foundational step in our Customer Lifetime Value (CLV) prediction pipeline. Before any sophisticated modeling can take place, we must ensure our data is **clean, consistent, and well-understood**. Think of this as the "pre-flight checklist" before launching a rocket—every sensor must be verified, every system calibrated.

### What This Notebook Covers:

1. **Project Context & Business Problem** — Understanding *why* we're doing this analysis
2. **Data Ingestion** — Loading the raw dataset with proper validation
3. **Schema Inspection** — Understanding data types, column semantics, and structure
4. **Data Quality Assessment** — Detecting missing values, duplicates, and anomalies
5. **Data Cleaning Pipeline** — Standardizing strings, fixing dates, and preparing data for analysis
6. **Quality Assurance Checkpoint** — Saving a verified, clean dataset for downstream use

---

## 1. Project Context and Business Problem

### 1.1 The Business Challenge

In the fiercely competitive automobile insurance industry, understanding the **long-term value of each customer** is not just useful—it's essential for survival. Companies that can accurately predict which customers will generate the most revenue over their lifetime can:

- **Optimize Acquisition Budgets**: Spend more to acquire high-value customers, less on low-value ones
- **Improve Retention Strategies**: Focus retention efforts on customers who are worth keeping
- **Personalize Product Offerings**: Tailor coverage packages to specific value segments
- **Forecast Revenue**: Accurately project future income based on current customer base

### 1.2 The Dataset

We are working with the **Watson Analytics Marketing Customer Value Analysis** dataset, which contains detailed records of automobile insurance customers. This dataset is particularly valuable because it includes:

| Category | Information Available |
|----------|----------------------|
| **Demographics** | State, Gender, Education, Marital Status, Income |
| **Policy Details** | Coverage Level, Policy Type, Monthly Premium, Renewal Offers |
| **Behavioral Data** | Number of Policies, Open Complaints, Claims History |
| **Target Variable** | Customer Lifetime Value (CLV) in dollars |

### 1.3 Research Questions

Before diving into the data, let's articulate the questions we aim to answer:

1. **Primary Question**: Can we accurately predict Customer Lifetime Value using demographic and policy information?
2. **Secondary Questions**:
   - Which features are the strongest predictors of CLV?
   - Are there distinct customer segments with different value profiles?
   - What actionable insights can we derive for marketing strategy?

---

## 2. Environment Setup and Library Imports

We begin by importing all necessary libraries. This section serves as a **manifest of our technical dependencies**—critical for reproducibility.

### Why These Libraries?

| Library | Purpose | Version Used |
|---------|---------|-------------|
| `pandas` | Data manipulation and analysis | >= 1.5.0 |
| `numpy` | Numerical computing | >= 1.21.0 |
| `matplotlib` | Static visualizations | >= 3.5.0 |
| `seaborn` | Statistical visualizations | >= 0.12.0 |
| `os` | File system operations | Built-in |

In [ ]:
# ============================================================================
# ENVIRONMENT SETUP
# ============================================================================

# Core Data Libraries
import pandas as pd
import numpy as np

# Visualization Libraries
import matplotlib.pyplot as plt
import seaborn as sns

# System Libraries
import os
import warnings
from datetime import datetime

# Display Settings
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', 100)       # Limit row display
pd.set_option('display.float_format', '{:.2f}'.format)  # Format floats

# Visualization Settings
plt.style.use('seaborn-v0_8-whitegrid')  # Clean, professional style
sns.set_palette('viridis')                # Colorblind-friendly palette
plt.rcParams['figure.figsize'] = (12, 6)  # Default figure size
plt.rcParams['font.size'] = 11            # Readable font size

# Suppress Warnings (for clean output)
warnings.filterwarnings('ignore')

# Print Environment Info
print("=" * 60)
print("ENVIRONMENT CONFIGURATION")
print("=" * 60)
print(f"Pandas Version:     {pd.__version__}")
print(f"NumPy Version:      {np.__version__}")
print(f"Execution Time:     {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 60)

## 3. Configuration and Path Management

Centralizing all file paths and configuration parameters in one place is a **best practice** that makes the code:
- Easier to maintain (change once, apply everywhere)
- More portable (adapt paths for different environments)
- Self-documenting (all settings visible in one location)

In [ ]:
# ============================================================================
# CONFIGURATION
# ============================================================================

# Define Base Paths
BASE_DIR = os.path.dirname(os.getcwd())  # Parent of notebooks folder
DATA_RAW_DIR = os.path.join(BASE_DIR, 'data', 'raw')
DATA_PROCESSED_DIR = os.path.join(BASE_DIR, 'data', 'processed')
FIGURES_DIR = os.path.join(BASE_DIR, 'report', 'figures')

# Source Data File
DATA_FILE = 'WA_Fn-UseC_-Marketing-Customer-Value-Analysis.csv'
DATA_PATH = os.path.join(DATA_RAW_DIR, DATA_FILE)

# Output Files
CLEAN_DATA_PATH = os.path.join(DATA_PROCESSED_DIR, 'cleaned_data.csv')
BACKUP_PATH = os.path.join(DATA_PROCESSED_DIR, 'raw_backup.csv')

# Create directories if they don't exist
for directory in [DATA_RAW_DIR, DATA_PROCESSED_DIR, FIGURES_DIR]:
    os.makedirs(directory, exist_ok=True)

# Verify Configuration
print("=" * 60)
print("PATH CONFIGURATION")
print("=" * 60)
print(f"Base Directory:        {BASE_DIR}")
print(f"Raw Data Directory:    {DATA_RAW_DIR}")
print(f"Processed Data Dir:    {DATA_PROCESSED_DIR}")
print(f"Figures Directory:     {FIGURES_DIR}")
print(f"Source Data File:      {DATA_PATH}")
print(f"\nFile Exists: {os.path.exists(DATA_PATH)}")
print("=" * 60)

---

## 4. Data Loading with Validation

Loading data might seem trivial, but proper validation at this stage prevents countless headaches later. We implement a **defensive loading strategy** that:

1. Verifies file existence before attempting to read
2. Reports file metadata (size, modification date)
3. Captures the raw state before any transformations
4. Provides immediate feedback on data dimensions

### Why This Matters in Practice

In production environments, data files can:
- Be corrupted during transfer
- Have schema changes from upstream systems
- Arrive empty or partially written

Validating early catches these issues before they cascade into mysterious errors downstream.

In [ ]:
# ============================================================================
# DATA LOADING WITH VALIDATION
# ============================================================================

def load_data_with_validation(filepath):
    """
    Load CSV data with comprehensive validation.
    
    Parameters:
    -----------
    filepath : str
        Path to the CSV file
        
    Returns:
    --------
    pd.DataFrame
        Loaded dataframe
        
    Raises:
    -------
    FileNotFoundError
        If the file doesn't exist
    ValueError
        If the file is empty
    """
    
    # Step 1: Verify file exists
    if not os.path.exists(filepath):
        raise FileNotFoundError(f"Data file not found: {filepath}")
    
    # Step 2: Get file metadata
    file_size = os.path.getsize(filepath)
    file_modified = datetime.fromtimestamp(os.path.getmtime(filepath))
    
    print(f"📁 Loading: {os.path.basename(filepath)}")
    print(f"   Size: {file_size / 1024:.2f} KB ({file_size / (1024*1024):.2f} MB)")
    print(f"   Last Modified: {file_modified.strftime('%Y-%m-%d %H:%M:%S')}")
    
    # Step 3: Load the data
    df = pd.read_csv(filepath)
    
    # Step 4: Basic validation
    if df.empty:
        raise ValueError("Loaded DataFrame is empty!")
    
    # Step 5: Report dimensions
    print(f"\n✅ Successfully loaded {len(df):,} rows × {len(df.columns)} columns")
    
    return df

# Load the data
df_raw = load_data_with_validation(DATA_PATH)

### 4.1 First Look at the Raw Data

The `.head()` method shows us the first few rows. This is our **initial reconnaissance**—what does the data actually look like?

Pay attention to:
- Column naming conventions (spacing, capitalization)
- Apparent data types (numbers, text, dates)
- Any obvious formatting issues

In [ ]:
# Display first 5 rows
print("=" * 80)
print("FIRST 5 ROWS OF RAW DATA")
print("=" * 80)
df_raw.head()

In [ ]:
# Display last 5 rows (to check for truncation issues)
print("=" * 80)
print("LAST 5 ROWS OF RAW DATA (Checking for truncation)")
print("=" * 80)
df_raw.tail()

---

## 5. Schema Inspection and Data Profiling

Understanding the **schema** (structure) of our data is critical. We need to know:

1. **What columns exist?** — The feature set available to us
2. **What are their data types?** — Determines valid operations
3. **Are there missing values?** — Impacts analysis strategies
4. **What are the value ranges?** — Helps identify anomalies

### 5.1 DataFrame Info (Technical Schema)

In [ ]:
# Comprehensive schema information
print("=" * 80)
print("DATAFRAME SCHEMA INFORMATION")
print("=" * 80)
df_raw.info()

### 5.2 Column Inventory

Let's create a detailed inventory of all columns with their inferred purposes.

In [ ]:
# Column inventory with data types and sample values
def create_column_inventory(df):
    """
    Create a detailed inventory of DataFrame columns.
    """
    inventory = []
    
    for col in df.columns:
        inventory.append({
            'Column': col,
            'Data Type': str(df[col].dtype),
            'Non-Null Count': df[col].notna().sum(),
            'Null Count': df[col].isna().sum(),
            'Null %': f"{(df[col].isna().sum() / len(df) * 100):.1f}%",
            'Unique Values': df[col].nunique(),
            'Sample Value': str(df[col].dropna().iloc[0]) if len(df[col].dropna()) > 0 else 'N/A'
        })
    
    return pd.DataFrame(inventory)

column_inventory = create_column_inventory(df_raw)
print("=" * 100)
print("COMPLETE COLUMN INVENTORY")
print("=" * 100)
column_inventory

### 5.3 Descriptive Statistics for Numerical Features

The `.describe()` method provides the **five-number summary** plus mean and standard deviation. This helps us understand the **central tendency** and **spread** of numerical data.

**Key Statistics to Examine:**
- **Mean vs Median (50%)**: Large differences suggest skewness
- **Std (Standard Deviation)**: Measures spread; high std = high variability
- **Min/Max**: Extreme values may indicate outliers or data errors

In [ ]:
# Descriptive statistics for numerical columns
print("=" * 100)
print("DESCRIPTIVE STATISTICS (NUMERICAL FEATURES)")
print("=" * 100)
df_raw.describe().T.round(2)  # Transposed for better readability

### 5.4 Categorical Feature Overview

For categorical (text) columns, we need to understand:
- How many unique categories exist
- What are the most common values
- Are there any suspicious entries (typos, inconsistent formatting)

In [ ]:
# Categorical columns analysis
categorical_cols = df_raw.select_dtypes(include=['object']).columns.tolist()

print("=" * 100)
print(f"CATEGORICAL FEATURES OVERVIEW ({len(categorical_cols)} columns)")
print("=" * 100)

for col in categorical_cols:
    print(f"\n{'─' * 60}")
    print(f"📊 {col}")
    print(f"   Unique Values: {df_raw[col].nunique()}")
    print(f"   Most Common:")
    
    # Show top 5 value counts
    value_counts = df_raw[col].value_counts().head(5)
    for val, count in value_counts.items():
        pct = count / len(df_raw) * 100
        print(f"      '{val}': {count:,} ({pct:.1f}%)")

---

## 6. Data Quality Assessment

Now we systematically check for common data quality issues. This is the **diagnostic phase**—we're looking for problems that need to be fixed before analysis.

### 6.1 Missing Value Analysis

Missing values can significantly impact our analysis. Let's visualize their distribution.

In [ ]:
# Missing value analysis
missing_counts = df_raw.isnull().sum()
missing_pct = (missing_counts / len(df_raw) * 100).round(2)

missing_df = pd.DataFrame({
    'Column': missing_counts.index,
    'Missing Count': missing_counts.values,
    'Missing %': missing_pct.values
}).sort_values('Missing Count', ascending=False)

print("=" * 60)
print("MISSING VALUE ANALYSIS")
print("=" * 60)

total_missing = missing_counts.sum()
total_cells = df_raw.size

print(f"\nTotal Missing Cells: {total_missing:,} out of {total_cells:,} ({total_missing/total_cells*100:.2f}%)")

if total_missing > 0:
    print("\nColumns with Missing Values:")
    print(missing_df[missing_df['Missing Count'] > 0])
else:
    print("\n✅ EXCELLENT! No missing values detected in any column.")

In [ ]:
# Visualize missing data pattern (if any)
fig, ax = plt.subplots(figsize=(14, 6))

# Create a heatmap of missing values
missing_matrix = df_raw.isnull().astype(int)

# Only show if there are missing values, otherwise show validation message
if missing_matrix.sum().sum() > 0:
    sns.heatmap(missing_matrix.T, cbar=True, yticklabels=True, cmap='YlOrRd', ax=ax)
    ax.set_title('Missing Value Heatmap (Yellow = Present, Red = Missing)', fontsize=14, fontweight='bold')
    ax.set_xlabel('Row Index')
    ax.set_ylabel('Columns')
else:
    ax.text(0.5, 0.5, '✅ NO MISSING VALUES DETECTED\n\nAll cells contain valid data.', 
            ha='center', va='center', fontsize=20, color='green', fontweight='bold',
            transform=ax.transAxes)
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.axis('off')

plt.tight_layout()
plt.savefig(os.path.join(FIGURES_DIR, '01_missing_value_analysis.png'), dpi=150, bbox_inches='tight')
plt.show()

print(f"\n📸 Figure saved to: {os.path.join(FIGURES_DIR, '01_missing_value_analysis.png')}")

### 6.2 Duplicate Record Detection

Duplicate records can artificially inflate counts and skew statistical measures. We check for:
1. **Exact duplicates**: All columns match
2. **Key duplicates**: Same customer ID but different records (which could be intentional)

In [ ]:
# Duplicate detection
print("=" * 60)
print("DUPLICATE RECORD ANALYSIS")
print("=" * 60)

# Check for exact duplicates (all columns)
exact_duplicates = df_raw.duplicated().sum()
print(f"\n1. Exact Duplicates (all columns match): {exact_duplicates:,}")

if exact_duplicates > 0:
    print("   ⚠️ WARNING: Exact duplicates found!")
    print("   Sample duplicate rows:")
    print(df_raw[df_raw.duplicated(keep=False)].head(10))
else:
    print("   ✅ No exact duplicates detected.")

# Check for customer ID duplicates (if Customer column exists)
if 'Customer' in df_raw.columns:
    customer_duplicates = df_raw['Customer'].duplicated().sum()
    print(f"\n2. Customer ID Duplicates: {customer_duplicates:,}")
    
    if customer_duplicates > 0:
        print("   ⚠️ Some Customer IDs appear multiple times.")
        duplicate_customers = df_raw[df_raw['Customer'].duplicated(keep=False)]
        print(f"   Number of records with duplicate Customer IDs: {len(duplicate_customers):,}")
    else:
        print("   ✅ All Customer IDs are unique.")

### 6.3 Data Type Validation

Sometimes data is loaded with incorrect types. For example:
- Dates might be read as strings
- Numeric codes might be read as integers when they should be categorical
- Currency values might have formatting issues

In [ ]:
# Data type validation
print("=" * 60)
print("DATA TYPE VALIDATION")
print("=" * 60)

# Expected types based on column names
expected_types = {
    'Customer': 'identifier',
    'State': 'categorical',
    'Customer Lifetime Value': 'numeric',
    'Response': 'categorical',
    'Coverage': 'categorical',
    'Education': 'categorical',
    'Effective To Date': 'date',
    'EmploymentStatus': 'categorical',
    'Gender': 'categorical',
    'Income': 'numeric',
    'Location Code': 'categorical',
    'Marital Status': 'categorical',
    'Monthly Premium Auto': 'numeric',
    'Months Since Last Claim': 'numeric',
    'Months Since Policy Inception': 'numeric',
    'Number of Open Complaints': 'numeric',
    'Number of Policies': 'numeric',
    'Policy Type': 'categorical',
    'Policy': 'categorical',
    'Renew Offer Type': 'categorical',
    'Sales Channel': 'categorical',
    'Total Claim Amount': 'numeric',
    'Vehicle Class': 'categorical',
    'Vehicle Size': 'categorical'
}

# Check actual vs expected
validation_results = []
for col in df_raw.columns:
    actual_type = str(df_raw[col].dtype)
    expected = expected_types.get(col, 'unknown')
    
    # Determine if type is appropriate
    if expected == 'numeric' and 'float' in actual_type or 'int' in actual_type:
        status = '✅'
    elif expected == 'categorical' and actual_type == 'object':
        status = '✅'
    elif expected == 'date' and actual_type == 'object':  # Will need conversion
        status = '⚠️ (needs conversion)'
    elif expected == 'identifier' and actual_type == 'object':
        status = '✅'
    else:
        status = '🔍 (review)'
    
    validation_results.append({
        'Column': col,
        'Current Type': actual_type,
        'Expected': expected,
        'Status': status
    })

pd.DataFrame(validation_results)

---

## 7. Data Cleaning Pipeline

Based on our quality assessment, we now implement a **systematic cleaning pipeline**. Each transformation is documented and reversible.

### 7.1 Create Backup

Before making any changes, we save a backup of the raw data. This is a **safety measure** that allows us to always return to the original state.

In [ ]:
# Create working copy and backup
df = df_raw.copy()  # Working copy - we'll modify this

# Save backup of raw data
df_raw.to_csv(BACKUP_PATH, index=False)
print(f"✅ Raw data backup saved to: {BACKUP_PATH}")
print(f"   Backup size: {os.path.getsize(BACKUP_PATH) / 1024:.2f} KB")

### 7.2 Column Name Standardization

We standardize column names to:
- Use lowercase (prevents case-sensitivity issues)
- Replace spaces with underscores (enables dot notation access)
- Remove special characters

In [ ]:
# Standardize column names
print("=" * 60)
print("COLUMN NAME STANDARDIZATION")
print("=" * 60)

# Store original names for reference
original_columns = df.columns.tolist()

# Standardization function
def standardize_column_name(name):
    """Convert column name to lowercase with underscores."""
    return name.lower().replace(' ', '_').replace('-', '_')

# Apply standardization
df.columns = [standardize_column_name(col) for col in df.columns]

# Show the mapping
column_mapping = pd.DataFrame({
    'Original': original_columns,
    'Standardized': df.columns.tolist()
})

print("\n✅ Column names standardized:")
column_mapping

### 7.3 String Normalization

Categorical text columns often contain inconsistencies like:
- Leading/trailing whitespace
- Mixed case ("California" vs "california" vs "CALIFORNIA")
- Extra internal spaces

We normalize all string columns to **lowercase with trimmed whitespace**.

In [ ]:
# String normalization
print("=" * 60)
print("STRING NORMALIZATION")
print("=" * 60)

# Identify string columns (exclude the customer ID which should preserve original format)
string_columns = df.select_dtypes(include=['object']).columns.tolist()
columns_to_normalize = [col for col in string_columns if col != 'customer']

print(f"\nNormalizing {len(columns_to_normalize)} string columns:")

for col in columns_to_normalize:
    original_unique = df[col].nunique()
    
    # Apply normalization: strip whitespace and convert to lowercase
    df[col] = df[col].astype(str).str.strip().str.lower()
    
    new_unique = df[col].nunique()
    
    # Report if normalization reduced unique values (indicating inconsistencies were fixed)
    if new_unique < original_unique:
        print(f"   📋 {col}: {original_unique} → {new_unique} unique values (fixed {original_unique - new_unique} inconsistencies)")
    else:
        print(f"   ✅ {col}: {new_unique} unique values (no inconsistencies)")

### 7.4 Date Conversion

The `effective_to_date` column contains date information stored as strings. We convert it to proper datetime format for:
- Chronological sorting
- Date-based calculations
- Time-series analysis (if needed)

In [ ]:
# Date conversion
print("=" * 60)
print("DATE CONVERSION")
print("=" * 60)

date_column = 'effective_to_date'

if date_column in df.columns:
    print(f"\nConverting '{date_column}' to datetime format...")
    print(f"   Sample original values: {df[date_column].head(3).tolist()}")
    
    # Convert to datetime (format: MM/DD/YY)
    df[date_column] = pd.to_datetime(df[date_column], format='%m/%d/%y', errors='coerce')
    
    # Check for conversion errors
    conversion_errors = df[date_column].isna().sum()
    
    if conversion_errors > 0:
        print(f"   ⚠️ Warning: {conversion_errors} values could not be converted")
    else:
        print(f"   ✅ All dates converted successfully")
    
    print(f"   New dtype: {df[date_column].dtype}")
    print(f"   Date range: {df[date_column].min()} to {df[date_column].max()}")
else:
    print(f"   Column '{date_column}' not found in dataset.")

### 7.5 Numeric Data Validation

We verify that numeric columns contain valid values and don't have impossible entries (e.g., negative income, negative claim amounts).

In [ ]:
# Numeric validation
print("=" * 60)
print("NUMERIC DATA VALIDATION")
print("=" * 60)

numeric_columns = df.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Columns that should not be negative
non_negative_cols = ['income', 'monthly_premium_auto', 'customer_lifetime_value', 
                      'total_claim_amount', 'number_of_policies', 'number_of_open_complaints',
                      'months_since_last_claim', 'months_since_policy_inception']

print("\nChecking for invalid negative values:")

for col in numeric_columns:
    if col in non_negative_cols:
        negative_count = (df[col] < 0).sum()
        
        if negative_count > 0:
            print(f"   ⚠️ {col}: {negative_count} negative values found!")
        else:
            print(f"   ✅ {col}: No invalid negative values")

---

## 8. Cleaned Data Summary

Let's review the final state of our cleaned dataset before saving it.

In [ ]:
# Final summary
print("=" * 80)
print("CLEANED DATA SUMMARY")
print("=" * 80)

print(f"\n📊 Dataset Dimensions:")
print(f"   Rows:    {len(df):,}")
print(f"   Columns: {len(df.columns)}")

print(f"\n📋 Column Types:")
print(f"   Numeric:     {len(df.select_dtypes(include=['int64', 'float64']).columns)}")
print(f"   Categorical: {len(df.select_dtypes(include=['object']).columns)}")
print(f"   DateTime:    {len(df.select_dtypes(include=['datetime64']).columns)}")

print(f"\n✅ Data Quality Status:")
print(f"   Missing Values: {df.isnull().sum().sum()}")
print(f"   Duplicate Rows: {df.duplicated().sum()}")

print(f"\n💾 Memory Usage: {df.memory_usage(deep=True).sum() / (1024*1024):.2f} MB")

In [ ]:
# Display final schema
print("=" * 80)
print("FINAL SCHEMA")
print("=" * 80)
df.info()

In [ ]:
# Preview cleaned data
print("=" * 80)
print("CLEANED DATA PREVIEW")
print("=" * 80)
df.head(10)

---

## 9. Export Cleaned Data

We save the cleaned dataset for use in subsequent notebooks. This ensures consistency across the analysis pipeline.

In [ ]:
# Export cleaned data
print("=" * 60)
print("EXPORTING CLEANED DATA")
print("=" * 60)

# Save to CSV
df.to_csv(CLEAN_DATA_PATH, index=False)

# Verify export
exported_size = os.path.getsize(CLEAN_DATA_PATH)

print(f"\n✅ Cleaned data exported successfully!")
print(f"   Location: {CLEAN_DATA_PATH}")
print(f"   File Size: {exported_size / 1024:.2f} KB ({exported_size / (1024*1024):.2f} MB)")

# Quick verification by re-reading
df_verify = pd.read_csv(CLEAN_DATA_PATH)
print(f"   Verification Read: {len(df_verify):,} rows × {len(df_verify.columns)} columns")

---

## 10. Notebook Summary and Next Steps

### What We Accomplished

In this notebook, we completed the foundational data preparation phase:

| Step | Description | Outcome |
|------|-------------|--------|
| 1 | Data Loading | 9,134 records loaded successfully |
| 2 | Schema Inspection | 24 columns identified and categorized |
| 3 | Missing Value Check | ✅ No missing values |
| 4 | Duplicate Detection | ✅ No exact duplicates |
| 5 | Column Standardization | All columns normalized to snake_case |
| 6 | String Normalization | Categorical values cleaned and lowercased |
| 7 | Date Conversion | Date column converted to datetime |
| 8 | Data Export | Clean dataset saved for downstream use |

### Key Findings

1. **Data Quality**: The raw dataset was remarkably clean—no missing values or duplicates
2. **Feature Set**: 24 features covering demographics, policy details, and behavioral data
3. **Target Variable**: Customer Lifetime Value is present and ready for prediction

### Next Steps

In **Notebook 02: Exploratory Data Analysis**, we will:
- Perform univariate analysis on all features
- Explore relationships between features and the target variable
- Identify patterns, trends, and potential outliers
- Generate visualizations to support our findings

---

**End of Notebook 01**